In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

In [2]:
# Load the dataset
data_url = "https://raw.githubusercontent.com/DwaipayanDutta/Assignment/main/Data%20Files/train.csv"  
labels_url = "https://raw.githubusercontent.com/DwaipayanDutta/Assignment/main/Data%20Files/train_churn_labels.csv"  
data = pd.read_csv(data_url)
labels = pd.read_csv(labels_url)

In [3]:
data

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,NaN,NaN,NaN,NaN,NaN,1526.0,7.0,NaN,NaN,NaN,...,oslk,fXVEsaq,jySVZNlOJy,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,525.0,0.0,NaN,NaN,NaN,...,oslk,2Kb5FSF,LM8l689qOp,NaN,NaN,fKCe,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5236.0,7.0,NaN,NaN,NaN,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,NaN,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,oslk,CE7uk3u,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,1029.0,7.0,NaN,NaN,NaN,...,oslk,1J2cvxe,LM8l689qOp,NaN,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,NaN,NaN,NaN,NaN,NaN,357.0,0.0,NaN,NaN,NaN,...,oslk,EROH7Cg,LM8l689qOp,NaN,NaN,7FJQ,RAYp,F2FyR07IdsN7I,NaN,NaN
49996,NaN,NaN,NaN,NaN,NaN,1078.0,0.0,NaN,NaN,NaN,...,oslk,GfSQowC,LM8l689qOp,NaN,kG3k,FSa2,RAYp,55YFVY9,am7c,NaN
49997,NaN,NaN,NaN,NaN,NaN,2807.0,7.0,NaN,NaN,NaN,...,oslk,dh6qI2t,LM8l689qOp,NaN,ELof,fKCe,RAYp,TCU50_Yjmm6GIBZ0lL_,NaN,NaN
49998,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,oslk,2fF2Oqu,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN


In [4]:
labels

,Label
0,-1
1,1
2,-1
3,-1
4,-1
...,...
49995,-1
49996,-1
49997,-1
49998,-1
